# DeepLabCut Toolbox - Docker
https://github.com/DeepLabCut/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*

This notebook demonstrates the necessary steps to use DeepLabCut on your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the Docker container to:
- train a network
- evaluate a network
- analyze a novel video

This assumes you already have a project folder with labeled data! 


## Let's look at info about the Docker Environment:


In [ ]:
!nvcc --version

In [ ]:
import torch

# Let's make sure we see a GPU:
print(torch.__version__)
print(torch.cuda.is_available())

## Start here for training DeepLabCut and analyzing new videos!

In [ ]:
# GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
# This notebook is for you to train and run video analysis!
import os

os.environ["DLClight"] = "True"

In [ ]:
# now we are ready to train!
import deeplabcut

### change to your path:

In [ ]:
# change to yours!
path_config_file = '/home/mackenzie/DEEPLABCUT/DeepLabCut/examples/Reaching-Mackenzie-2018-08-30/config.yaml'

## Note, if you moved the project, or downloaded this and using the demo code, you will need to edit the project path in the config.yaml file! 

Head over to the project folder to open the yaml file in any text editor (such as gedit in Ubuntu)

(description): project_path:  Full path of the project (edit if you need to move the project to a cluster/server/another computer or a different directory on your computer)

## Create a training dataset

This function generates the training data required for DeepLabCut. The user can set the fraction of the training set size (from all labeled images in the hd5 file) in the `config.yaml` file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and creates a `pytorch_config.yaml` file, defining the model architecture and containing various parameters used for training the network. For most all use cases we have seen, the defaults are perfectly fine. For more information about the variables that can be set, check out the [docs](https://deeplabcut.github.io/DeepLabCut/docs/pytorch/pytorch_config.html)!


In [ ]:
deeplabcut.create_training_dataset(path_config_file, Shuffles=[1])

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=2,
    displayiters=5,
)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it
# hits the end (default, 200 epochs).

# If you end training before it hits the end, you will see what looks like
# an error message, but it's not an error - don't worry....


## Start evaluating
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [ ]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only
# be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of DOCKER!** We recommend coming back to train and analyze videos... 
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
videofile_path = [
    "/home/mackenzie/DEEPLABCUT/DeepLabCut/examples/Reaching-Mackenzie-2018-08-30/videos/MovieS2_Perturbation_noLaser_compressed.avi"
]  # Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file, videofile_path)

## Create labeled video
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file, videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
# for making interactive plots.
# deeplabcut.plot_trajectories(path_config_file, videofile_path, plotting=True)
deeplabcut.plot_trajectories(path_config_file, videofile_path, showfigures=True)